In [0]:
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema2/ingestion_volume/target/custsmod.txt",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.show(20,False)
display(rawdf1.take(20))
display(rawdf1.sample(.1))

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema2/ingestion_volume/target/custsmod.txt",mode='permissive')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema2/ingestion_volume/target/custsmod.txt",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)
     

In [0]:
#method3 best methodology of applying active data munging
#Validation by doing cleansing (not at the time of creating Dataframe, rather we will clean and scrub subsequently)...
struttype1=StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema2/ingestion_volume/target/custsmod.txt",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
     

In [0]:
#Important na functions we can use to do cleansing
cleanseddf=rawdf1.na.drop(how="any")#This function will drop any column in a given row with null otherwise this function returns rows with no null columns
display(cleanseddf.where("age is null"))#any one column contains null will be cleaned
display(rawdf1.where("age is null"))
cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["lastname","profession"])#4000004,Gretchen,,66,
display(cleanseddf)

In [0]:

cleanseddf=rawdf1.na.fill('not provided',subset=["profession"])#4000004,Gretchen,,66,
display(cleanseddf)

4009866